In [2]:
from flask import Flask,jsonify,render_template
import json
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import datetime as dt

make flask server and do sql stuff

In [12]:
app = Flask(__name__)

with open("../restaurant_data.json", 'r') as file:
    data = json.load(file)
    

restaurant_data = data
# json.load("restaurant_data.json")
restaurnt_df = pd.DataFrame(restaurant_data)
# json.loads
#--SQL STUFF
# engine = create_engine("sqlite:///Resources/votes.sqlite")
# base = automap_base()
# base.prepare(autoload_with=engine)
#-- 

@app.route("/")
def home():
    print("YOLO CHOLO")
    # Start at the homepage.
    # List all the available routes.
    return (
        f"Welcome to the Portland Eats API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/home<br/>"
        f"/api/v1.0/map/<br/>"
        f"/api/v1.0/chart/<br/>"
        f"/api/v1.0/btc/<br/>"
        )



@app.route("/api/v1.0/home")
def wait():
    return render_template('index.html')
@app.route("/api/v1.0/map")
def dontwait():
    return render_template('map.html')
@app.route("/api/v1.0/chart")
def justkiddingyoucanwait():
    return render_template('chart.html')
@app.route("/api/v1.0/btc")
def nvm():
    return render_template('btc.html')

app.run()
# if(__name__=="__main__"):
#     app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2023 20:44:16] "GET /api/v1.0/voting2 HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 20:44:22] "GET /api/v1.0/home HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 20:44:22] "GET /api/v1.0/index.js HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2023 20:44:47] "GET /api/v1.0/btc HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2023 20:44:48] "GET /api/v1.0/index.js HTTP/1.1" 404 -
